In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os
os.getcwd()

'/Users/chloebergsma-safar/Desktop/Metis/NBM_Regression_Project/Metis-Regression'

## Read in data
- schools_csv.csv contains first 400 schools
- schools_csv_2.csv contains the second 400 schools
- schools_csv_3.csv contains the third 400 schools
- schools_csv_4.csv contains the last 132 schools
- total # of schools: 1332

In [3]:
schools_1_400 = pd.read_csv('schools_csv.csv')

In [4]:
schools_401_800 = pd.read_csv('schools_csv_2.csv')

In [5]:
schools_801_1200 = pd.read_csv('schools_csv_3.csv')

In [6]:
schools_1201_1332 = pd.read_csv('schools_csv_4.csv')

## Concatenate dataframes

In [86]:
schools_df = pd.concat([schools_1_400, schools_401_800, schools_801_1200, schools_1201_1332], axis=0)

In [8]:
schools_1_400.shape

(400, 36)

### Check concatenation

In [9]:
schools_401_800.shape

(400, 36)

In [10]:
schools_801_1200.shape

(400, 36)

In [11]:
schools_1201_1332.shape

(132, 36)

In [12]:
schools_df.shape

(1332, 36)

## Peform EDA on concatenated dataframe

In [13]:
schools_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1332 entries, 0 to 131
Data columns (total 36 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   school                                   1332 non-null   object 
 1   raw_stu_to_fac_ratio                     1323 non-null   object 
 2   tuition_fees_19_20                       1261 non-null   float64
 3   books_supplies_19_20                     1209 non-null   float64
 4   tuition_fees                             1264 non-null   float64
 5   books_supplies                           1214 non-null   float64
 6   tuition                                  48 non-null     float64
 7   fees                                     48 non-null     float64
 8   tuition_under                            11 non-null     float64
 9   fees_under                               11 non-null     float64
 10  ft_faculty                               1316 non

### Clean up tuition and fees columns

#### Consolidate tuition and tuition_under columns, since, based on webscraping, we know that they represent the same data stored in different places on school pages (thus, the decision to extract the data differently)

In [87]:
schools_df['tuition_20_21'] = np.nan

In [88]:
schools_df.tuition_20_21.fillna(schools_df.tuition, inplace=True)

In [89]:
schools_df.tuition_20_21.fillna(schools_df.tuition_under, inplace=True)

In [90]:
# check that in new col, where tuition has a value, it was assigned to the new col
tuition_not_nan = schools_df.loc[~schools_df.tuition.isna()]
tuition_not_nan[['tuition_20_21', 'tuition', 'tuition_under']].head()

,tuition_20_21,tuition,tuition_under
4,20370.0,20370.0,NaN
5,17820.0,17820.0,NaN
6,21600.0,21600.0,NaN
9,26820.0,26820.0,NaN
10,9375.0,9375.0,NaN


In [91]:
# check that in new col, where tuition_under has a value, it was assigned to the new col
tuition_under_not_nan = schools_df.loc[~schools_df.tuition_under.isna()]
tuition_under_not_nan[['tuition_20_21', 'tuition', 'tuition_under']].head()

,tuition_20_21,tuition,tuition_under
13,22230.0,NaN,22230.0
251,13440.0,NaN,13440.0
18,10440.0,NaN,10440.0
70,26400.0,NaN,26400.0
82,19975.0,NaN,19975.0


In [92]:
# drop tuition and tuition_under columns
schools_df.drop(['tuition', 'tuition_under'],axis=1, inplace=True)

#### Consolidate fees and fees_under columns, since, based on webscraping, we know that they represent the same data stored in different places on school pages (thus, the decision to extract the data differently)

In [93]:
schools_df['fees_20_21'] = np.nan

In [94]:
schools_df.fees_20_21.fillna(schools_df.fees, inplace=True)

In [95]:
schools_df.fees_20_21.fillna(schools_df.fees_under, inplace=True)

In [96]:
# check that in new col, where fees has a value, it was assigned to the new col
fees_not_nan = schools_df.loc[~schools_df.fees.isna()]
fees_not_nan[['fees_20_21', 'fees', 'fees_under']].head()

,fees_20_21,fees,fees_under
4,300.0,300.0,NaN
5,300.0,300.0,NaN
6,435.0,435.0,NaN
9,75.0,75.0,NaN
10,600.0,600.0,NaN


In [97]:
# check that in new col, where fees_under has a value, it was assigned to the new col
fees_under_not_nan = schools_df.loc[~schools_df.fees_under.isna()]
fees_under_not_nan[['fees_20_21', 'fees', 'fees_under']].head()

,fees_20_21,fees,fees_under
13,1644.0,NaN,1644.0
251,0.0,NaN,0.0
18,60.0,NaN,60.0
70,900.0,NaN,900.0
82,2100.0,NaN,2100.0


In [98]:
# drop fees and fees_under columns
schools_df.drop(['fees', 'fees_under'],axis=1, inplace=True)

#### Combine new tuition_20_21 and fees_20_21 columns

In [99]:
schools_df['tuition_fees_20_21'] = schools_df.tuition_20_21 + schools_df.fees_20_21

In [100]:
# check that in new col, where tuition_20_21 and fees_20_21 have values, the sum
    # was assigned to the new column
tuition_20_21_not_nan = schools_df.loc[~(schools_df.tuition_20_21.isna())]
tuition_20_21_not_nan[['tuition_fees_20_21', 'tuition_20_21', 'fees_20_21']].tail()

,tuition_fees_20_21,tuition_20_21,fees_20_21
79,78241.0,73200.0,5041.0
89,13386.0,12984.0,402.0
101,26200.0,26200.0,0.0
109,18000.0,18000.0,0.0
128,10625.0,10000.0,625.0


#### Consolidate tuition_fees_20_21 and tuition_fees columns since, based on webscraping, we know that they represent the same data stored in different places on school pages (thus, the decision to extract the data differently)

In [101]:
schools_df['tuition_fees_20_21_fin'] = np.nan

In [102]:
schools_df.tuition_fees_20_21_fin.fillna(schools_df.tuition_fees, inplace=True)

In [103]:
schools_df.tuition_fees_20_21_fin.fillna(schools_df.tuition_fees_20_21, inplace=True)

In [104]:
# check that in new col, where tuition_fees has a value, it was assigned to the new col
tuition_fees_not_nan = schools_df.loc[~schools_df.tuition_fees.isna()]
tuition_fees_not_nan[['tuition_fees_20_21_fin', 'tuition_fees_20_21', 'tuition_fees']].head()

,tuition_fees_20_21_fin,tuition_fees_20_21,tuition_fees
0,9700.0,NaN,9700.0
1,20760.0,NaN,20760.0
2,6250.0,NaN,6250.0
3,32404.0,NaN,32404.0
7,46486.0,NaN,46486.0


In [105]:
# check that in new col, where tuition_fees_20_21 has a value, it was assigned to the new col
tuition_fees_20_21_not_nan = schools_df.loc[~schools_df.tuition_fees_20_21.isna()]
tuition_fees_20_21_not_nan[['tuition_fees_20_21_fin', 'tuition_fees_20_21', 'tuition_fees']].head()

,tuition_fees_20_21_fin,tuition_fees_20_21,tuition_fees
4,20670.0,20670.0,NaN
5,18120.0,18120.0,NaN
6,22035.0,22035.0,NaN
9,26895.0,26895.0,NaN
10,9975.0,9975.0,NaN


In [107]:
# drop tuition_fees, tuition_fees_20_21, fees_20_21 and tuition_20_21 columns
schools_df.drop(['tuition_fees', 'tuition_fees_20_21', 'fees_20_21', 'tuition_20_21'],axis=1, inplace=True)

#### Drop tuition_fees_19_20 column and any rows with nan in tuition_fees_20_21_fin (our target variable for the linear regression)

In [109]:
schools_df.drop(['tuition_fees_19_20'],axis=1, inplace=True)

In [111]:
schools_df.dropna(subset=['tuition_fees_20_21_fin'], inplace=True)

In [113]:
schools_df.shape # by dropping nan in tuition_fees_20_21_fin, we lost 9 rows

(1323, 31)

## Clean enrollment columns

In [119]:
schools_df[['school','total_enroll', 'total_enroll_under', 'under_enroll', 'under_trans_enroll', 'grad_enroll']].sample(10)

,school,total_enroll,total_enroll_under,under_enroll,under_trans_enroll,grad_enroll
249,Whittier College,1564.0,NaN,1490.0,59.0,74.0
72,Southeastern University,9546.0,NaN,8312.0,505.0,1234.0
173,Chestnut Hill College,1528.0,NaN,1074.0,72.0,454.0
34,St Luke's College,NaN,211.0,NaN,49.0,NaN
241,University of the West,238.0,NaN,114.0,11.0,124.0
94,Concordia University-Wisconsin,5492.0,NaN,3094.0,183.0,2398.0
16,Miles College,NaN,1440.0,NaN,84.0,NaN
239,University of Detroit Mercy,4987.0,NaN,2633.0,183.0,2354.0
94,Alvernia University,2560.0,NaN,2045.0,59.0,515.0
339,St. Thomas Aquinas College,1779.0,NaN,1612.0,68.0,167.0


In [120]:
schools_df['total_under_enroll'] = np.nan

In [121]:
schools_df['total_under_enroll'].fillna(schools_df.total_enroll_under, inplace=True)

In [122]:
schools_df['total_under_enroll'].fillna(schools_df.under_enroll, inplace=True)

In [123]:
# check that in new col, where total_enroll_under has a value, it was assigned to the new col
total_enroll_under_not_nan = schools_df.loc[~schools_df.total_enroll_under.isna()]
total_enroll_under_not_nan[['total_under_enroll', 'total_enroll_under', 'under_enroll']].head()

,total_under_enroll,total_enroll_under,under_enroll
0,40.0,40.0,NaN
13,2.0,2.0,NaN
14,3180.0,3180.0,NaN
15,101.0,101.0,NaN
17,44481.0,44481.0,NaN


In [124]:
# check that in new col, where under_enroll has a value, it was assigned to the new col
under_enroll_not_nan = schools_df.loc[~schools_df.under_enroll.isna()]
under_enroll_not_nan[['total_under_enroll', 'total_enroll_under', 'under_enroll']].head()

,total_under_enroll,total_enroll_under,under_enroll
1,312.0,NaN,312.0
2,65.0,NaN,65.0
3,37.0,NaN,37.0
4,169.0,NaN,169.0
5,66.0,NaN,66.0


In [115]:
schools_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1323 entries, 0 to 131
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   school                                   1323 non-null   object 
 1   raw_stu_to_fac_ratio                     1318 non-null   object 
 2   books_supplies_19_20                     1206 non-null   float64
 3   books_supplies                           1211 non-null   float64
 4   ft_faculty                               1309 non-null   float64
 5   pt_faculty                               1309 non-null   float64
 6   avg_amt_grant_schol_aid_beg_under_19_20  1276 non-null   float64
 7   perc_grant_schol_aid_beg_under_19_20     1315 non-null   float64
 8   avg_amt_grant_schol_aid_all_under_19_20  1313 non-null   float64
 9   total_enroll                             990 non-null    float64
 10  total_enroll_under                       328 non-

#### Notes: 
- If perc_grant_schol_aid_beg_under_19_20 > 100, it means it's capturing # of students, not %
- if avg_amt_grant_schol_aid_beg_under_19_20 == avg_amt_grant_schol_aid_all_under_19_20, it means there's only data for 'All Undergraduate Students'